In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

### TopKPooling

In [27]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
from torch_geometric.datasets import GitHub
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
max_nodes = 150
dataset_sparse = PygNodePropPredDataset(root='/data/XXX/Pooling', name="ogbn-arxiv")
evaluator = Evaluator("ogbn-arxiv")
eval_metric = dataset_sparse.eval_metric

In [9]:
from torch_geometric.datasets import GitHub
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_TopK(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_TopK, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(TopKPooling(channels, ratio=0.9))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm, _ = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
val_acc_variances = []
test_acc_variances = []
time_variances = []
memory_variances = []
gpu_memory_variances = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_TopK(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
val_acc_variance = np.var([result['best_val_acc'] for result in results])
test_acc_variance = np.var([result['corresponding_test_acc'] for result in results])
time_variance = np.var(times)
memory_variance = np.var(memories)
gpu_memory_variance = np.var(gpu_memories)
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")
print(f"\nValidation Accuracy Variance: {val_acc_variance:.4f}")
print(f"Test Accuracy Variance: {test_acc_variance:.4f}")
print(f"Time Variance: {time_variance:.4f}")
print(f"Memory Usage Variance: {memory_variance:.4f}MB")
print(f"GPU Memory Usage Variance: {gpu_memory_variance:.4f}MB")

Seed: 042, Epoch: 010, Loss: 3.2028, Train Acc: 0.1639, Val Acc: 0.2573, Test Acc: 0.2359
Seed: 042, Epoch: 020, Loss: 2.5912, Train Acc: 0.3391, Val Acc: 0.3953, Test Acc: 0.3635
Seed: 042, Epoch: 030, Loss: 2.1476, Train Acc: 0.4297, Val Acc: 0.4340, Test Acc: 0.3805
Seed: 042, Epoch: 040, Loss: 1.9460, Train Acc: 0.4819, Val Acc: 0.4755, Test Acc: 0.4163
Seed: 042, Epoch: 050, Loss: 1.7602, Train Acc: 0.5206, Val Acc: 0.5038, Test Acc: 0.4446
Seed: 042, Epoch: 060, Loss: 1.6680, Train Acc: 0.5400, Val Acc: 0.5195, Test Acc: 0.4594
Seed: 042, Epoch: 070, Loss: 1.6075, Train Acc: 0.5552, Val Acc: 0.5365, Test Acc: 0.4731
Seed: 042, Epoch: 080, Loss: 1.5767, Train Acc: 0.5618, Val Acc: 0.5450, Test Acc: 0.4829
Seed: 042, Epoch: 090, Loss: 1.5513, Train Acc: 0.5672, Val Acc: 0.5433, Test Acc: 0.4763
Seed: 042, Epoch: 100, Loss: 1.5279, Train Acc: 0.5714, Val Acc: 0.5533, Test Acc: 0.4947
Seed: 042, Epoch: 110, Loss: 1.5049, Train Acc: 0.5774, Val Acc: 0.5526, Test Acc: 0.4898
Seed: 042,

### SAGPooling

In [5]:
from torch_geometric.nn import SAGPooling
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_SAG(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_SAG, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(SAGPooling(channels, ratio=0.7))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm, _ = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_SAG(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

Seed: 042, Epoch: 010, Loss: 2.9403, Train Acc: 0.2903, Val Acc: 0.3134, Test Acc: 0.2990
Seed: 042, Epoch: 020, Loss: 2.4534, Train Acc: 0.3768, Val Acc: 0.4007, Test Acc: 0.3550
Seed: 042, Epoch: 030, Loss: 2.0780, Train Acc: 0.4409, Val Acc: 0.4381, Test Acc: 0.3763
Seed: 042, Epoch: 040, Loss: 1.8896, Train Acc: 0.4966, Val Acc: 0.4855, Test Acc: 0.4264
Seed: 042, Epoch: 050, Loss: 1.7429, Train Acc: 0.5291, Val Acc: 0.5180, Test Acc: 0.4522
Seed: 042, Epoch: 060, Loss: 1.6436, Train Acc: 0.5456, Val Acc: 0.5330, Test Acc: 0.4698
Seed: 042, Epoch: 070, Loss: 1.5837, Train Acc: 0.5602, Val Acc: 0.5428, Test Acc: 0.4767
Seed: 042, Epoch: 080, Loss: 1.5461, Train Acc: 0.5676, Val Acc: 0.5466, Test Acc: 0.4823
Seed: 042, Epoch: 090, Loss: 1.5217, Train Acc: 0.5741, Val Acc: 0.5520, Test Acc: 0.4874
Seed: 042, Epoch: 100, Loss: 1.5000, Train Acc: 0.5785, Val Acc: 0.5563, Test Acc: 0.4945
Seed: 042, Epoch: 110, Loss: 1.4831, Train Acc: 0.5831, Val Acc: 0.5609, Test Acc: 0.4980
Seed: 042,

### COPooling

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.
    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)
    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])
    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)
    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))
    return out
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero().view(-1)
        return perm
    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)
        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)
        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]
        return x_perm[batch_perm[mask]]
    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")
class GPR_prop(MessagePassing):
    '''
    propagation class for GPR_GNN
    '''
    def __init__(self, K, alpha, Init, Gamma=None, bias=True, **kwargs):
        super(GPR_prop, self).__init__(aggr='add', **kwargs)
        self.K = K
        self.Init = Init
        self.alpha = alpha
        assert Init in ['SGC', 'PPR', 'NPPR', 'Random', 'WS']
        if Init == 'SGC':
            TEMP = 0.0*np.ones(K+1)
            TEMP[alpha] = 1.0
        elif Init == 'PPR':
            TEMP = alpha*(1-alpha)**np.arange(K+1)
            TEMP[-1] = (1-alpha)**K
        elif Init == 'NPPR':
            TEMP = (alpha)**np.arange(K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'Random':
            bound = np.sqrt(3/(K+1))
            TEMP = np.random.uniform(-bound, bound, K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'WS':
            TEMP = Gamma
        self.temp = Parameter(torch.tensor(TEMP))
    def reset_parameters(self):
        torch.nn.init.zeros_(self.temp)
        for k in range(self.K+1):
            self.temp.data[k] = self.alpha*(1-self.alpha)**k
        self.temp.data[-1] = (1-self.alpha)**self.K
    def forward(self, x, edge_index, edge_weight=None):
        edge_index, norm = gcn_norm(
            edge_index, edge_weight, num_nodes=x.size(0), dtype=x.dtype)
        hidden = x*(self.temp[0])
        for k in range(self.K):
            x = self.propagate(edge_index, x=x, norm=norm)
            gamma = self.temp[k+1]
            hidden = hidden + gamma*x
        return hidden
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def __repr__(self):
        return '{}(K={}, temp={})'.format(self.__class__.__name__, self.K,
                                           self.temp)
class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes) 
        edge_index = edge_index.type(torch.long)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)
        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        return aggr_out
class graph_attention(torch.nn.Module):
    src_nodes_dim = 0  
    trg_nodes_dim = 1  
    nodes_dim = 0      
    head_dim = 1       
    def __init__(self, num_in_features, num_out_features, num_of_heads, dropout_prob=0.6, log_attention_weights=False):
        super().__init__()
        self.num_of_heads = num_of_heads
        self.num_out_features = num_out_features
        self.linear_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)
        self.scoring_fn_target = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.scoring_fn_source = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.init_params()
    def init_params(self):
        """
        The reason we're using Glorot (aka Xavier uniform) initialization is because it's a default TF initialization:
            https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow
        The original repo was developed in TensorFlow (TF) and they used the default initialization.
        Feel free to experiment - there may be better initializations depending on your problem.
        """
        nn.init.xavier_uniform_(self.linear_proj.weight)
        nn.init.xavier_uniform_(self.scoring_fn_target)
        nn.init.xavier_uniform_(self.scoring_fn_source)
    def forward(self, x, edge_index):
        in_nodes_features = x  
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]
        nodes_features_proj = self.linear_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        scores_per_edge = scores_source_lifted + scores_target_lifted
        return torch.sigmoid(scores_per_edge)
    def lift(self, scores_source, scores_target, nodes_features_matrix_proj, edge_index):
        """
        Lifts i.e. duplicates certain vectors depending on the edge index.
        One of the tensor dims goes from N -> E (that's where the "lift" comes from).
        """
        src_nodes_index = edge_index[self.src_nodes_dim]
        trg_nodes_index = edge_index[self.trg_nodes_dim]
        scores_source = scores_source.index_select(self.nodes_dim, src_nodes_index)
        scores_target = scores_target.index_select(self.nodes_dim, trg_nodes_index)
        nodes_features_matrix_proj_lifted = nodes_features_matrix_proj.index_select(self.nodes_dim, src_nodes_index)
        return scores_source, scores_target, nodes_features_matrix_proj_lifted
class CoPooling(torch.nn.Module):
    def __init__(self, ratio=0.5, K=0.05, edge_ratio=0.6, nhid=64, alpha=0.1, Init='Random', Gamma=None):
        super(CoPooling, self).__init__()
        self.ratio = ratio
        self.calc_information_score = NodeInformationScore()
        self.edge_ratio = edge_ratio
        self.prop1 = GPR_prop(K, alpha, Init, Gamma)
        score_dim = 32
        self.G_att = graph_attention(num_in_features=nhid, num_out_features=score_dim, num_of_heads=1)
        self.weight = Parameter(torch.Tensor(2*nhid, nhid))
        nn.init.xavier_uniform_(self.weight.data)
        self.bias = Parameter(torch.Tensor(nhid))
        nn.init.zeros_(self.bias.data)
        self.reset_parameters()
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight.data)
        nn.init.zeros_(self.bias.data)
        self.prop1.reset_parameters()
        self.G_att.init_params()
    def forward(self, x, edge_index, edge_attr, batch=None, nodes_index=None, node_attr=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        ori_batch = batch.clone()
        device = x.device
        num_nodes = x.shape[0]
        x_cut = self.prop1(x, edge_index) 
        attention = self.G_att(x_cut, edge_index) 
        attention = attention.sum(dim=1) 
        edge_index, attention = add_self_loops(edge_index, attention, 1.0, num_nodes) 
        edge_index_t, attention_t = transpose(edge_index, attention, num_nodes, num_nodes)
        edge_tmp = torch.cat((edge_index, edge_index_t), 1)
        att_tmp = torch.cat((attention, attention_t),0)
        edge_index, attention = coalesce(edge_tmp, att_tmp, num_nodes, num_nodes, 'mean')
        attention_np = attention.cpu().data.numpy()
        cut_val = np.percentile(attention_np, int(100*(1-self.edge_ratio))) 
        attention = attention * (attention >= cut_val) 
        kep_idx = attention > 0.0
        cut_edge_index, cut_edge_attr = edge_index[:, kep_idx], attention[kep_idx]
        x_information_score = self.calc_information_score(x, cut_edge_index, cut_edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        perm = topk(score, self.ratio, batch)
        x_topk = x[perm]
        batch = batch[perm]
        if nodes_index is not None:
            nodes_index = nodes_index[perm]
        if node_attr is not None:
            node_attr = node_attr[perm]
        if cut_edge_index is not None or cut_edge_index.nelement() != 0:
            induced_edge_index, induced_edge_attr = filter_adj(cut_edge_index, cut_edge_attr, perm, num_nodes=num_nodes)
        else:
            print('All edges are cut!')
            induced_edge_index, induced_edge_attr = cut_edge_index, cut_edge_attr
        attention_dense = (to_dense_adj(cut_edge_index, edge_attr=cut_edge_attr, max_num_nodes=num_nodes)).squeeze()
        x = F.relu(torch.matmul(torch.cat((x_topk, torch.matmul(attention_dense[perm],x)), 1), self.weight) + self.bias)
        return x, induced_edge_index, perm, induced_edge_attr, batch, nodes_index, node_attr, attention_dense

In [8]:
from torch_geometric.nn import ASAPooling
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_CO(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_CO, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(CoPooling(ratio=0.7, K=2, edge_ratio=0.6, nhid=256, alpha=0.1, Init='Random', Gamma=1.0))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, perm, _, batch, _, _, _ = self.pools[i - 1](x, edge_index, edge_attr=None, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_CO(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

OutOfMemoryError: CUDA out of memory. Tried to allocate 106.83 GiB. GPU 0 has a total capacty of 79.21 GiB of which 62.00 GiB is free. Process 686389 has 2.02 GiB memory in use. Process 273867 has 10.64 GiB memory in use. Including non-PyTorch memory, this process has 4.54 GiB memory in use. Of the allocated memory 2.86 GiB is allocated by PyTorch, and 333.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### CGIPooling

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
from torch_geometric.datasets import GitHub
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
max_nodes = 150
dataset_sparse = PygNodePropPredDataset(root='/data/XXX/Pooling', name="ogbn-arxiv")
evaluator = Evaluator("ogbn-arxiv")
eval_metric = dataset_sparse.eval_metric
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
from typing import Callable, Optional, Union
import torch
from torch import Tensor
@dataclass(init=False)
class SelectOutput:
    r"""The output of the :class:`Select` method, which holds an assignment
    from selected nodes to their respective cluster(s).
    Args:
        node_index (torch.Tensor): The indices of the selected nodes.
        num_nodes (int): The number of nodes.
        cluster_index (torch.Tensor): The indices of the clusters each node in
            :obj:`node_index` is assigned to.
        num_clusters (int): The number of clusters.
        weight (torch.Tensor, optional): A weight vector, denoting the strength
            of the assignment of a node to its cluster. (default: :obj:`None`)
    """
    node_index: Tensor
    num_nodes: int
    cluster_index: Tensor
    num_clusters: int
    weight: Optional[Tensor] = None
    def __init__(
        self,
        node_index: Tensor,
        num_nodes: int,
        cluster_index: Tensor,
        num_clusters: int,
        weight: Optional[Tensor] = None,
    ):
        if node_index.dim() != 1:
            raise ValueError(f"Expected 'node_index' to be one-dimensional "
                             f"(got {node_index.dim()} dimensions)")
        if cluster_index.dim() != 1:
            raise ValueError(f"Expected 'cluster_index' to be one-dimensional "
                             f"(got {cluster_index.dim()} dimensions)")
        if node_index.numel() != cluster_index.numel():
            raise ValueError(f"Expected 'node_index' and 'cluster_index' to "
                             f"hold the same number of values (got "
                             f"{node_index.numel()} and "
                             f"{cluster_index.numel()} values)")
        if weight is not None and weight.dim() != 1:
            raise ValueError(f"Expected 'weight' vector to be one-dimensional "
                             f"(got {weight.dim()} dimensions)")
        if weight is not None and weight.numel() != node_index.numel():
            raise ValueError(f"Expected 'weight' to hold {node_index.numel()} "
                             f"values (got {weight.numel()} values)")
        self.node_index = node_index
        self.num_nodes = num_nodes
        self.cluster_index = cluster_index
        self.num_clusters = num_clusters
        self.weight = weight
class Select(torch.nn.Module):
    r"""An abstract base class for implementing custom node selections as
    described in the `"Understanding Pooling in Graph Neural Networks"
    <https://arxiv.org/abs/1905.05178>`_ paper, which maps the nodes of an
    input graph to supernodes in the coarsened graph.
    Specifically, :class:`Select` returns a :class:`SelectOutput` output, which
    holds a (sparse) mapping :math:`\mathbf{C} \in {[0, 1]}^{N \times C}` that
    assigns selected nodes to one or more of :math:`C` super nodes.
    """
    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        pass
    def forward(self, *args, **kwargs) -> SelectOutput:
        raise NotImplementedError
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}()'
def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.
    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)
    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])
    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)
    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))
    return out
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero().view(-1)
        return perm
    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)
        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)
        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]
        return x_perm[batch_perm[mask]]
    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(in_channels * 2, in_channels)
        self.fc2 = nn.Linear(in_channels, 1)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.sigmoid(self.fc2(x))
        return x
class CGIPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, non_lin=torch.tanh):
        super(CGIPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.non_lin = non_lin
        self.hidden_dim = in_channels
        self.transform = GraphConv(in_channels, self.hidden_dim)
        self.pp_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.np_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.positive_pooling = GraphConv(self.hidden_dim, 1)
        self.negative_pooling = GraphConv(self.hidden_dim, 1)
        self.discriminator = Discriminator(self.hidden_dim)
        self.loss_fn = torch.nn.BCELoss()
    def forward(self, x, edge_index, edge_attr=None, batch=None):
        device = x.device  
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x_transform = F.leaky_relu(self.transform(x, edge_index), 0.2)
        x_tp = F.leaky_relu(self.pp_conv(x, edge_index), 0.2)
        x_tn = F.leaky_relu(self.np_conv(x, edge_index), 0.2)
        s_pp = self.positive_pooling(x_tp, edge_index).squeeze()
        s_np = self.negative_pooling(x_tn, edge_index).squeeze()
        perm_positive = topk(s_pp, 1, batch)
        perm_negative = topk(s_np, 1, batch)
        x_pp = x_transform[perm_positive] * self.non_lin(s_pp[perm_positive]).view(-1, 1)
        x_np = x_transform[perm_negative] * self.non_lin(s_np[perm_negative]).view(-1, 1)
        x_pp_readout = gap(x_pp, batch[perm_positive])
        x_np_readout = gap(x_np, batch[perm_negative])
        x_readout = gap(x_transform, batch)
        positive_pair = torch.cat([x_pp_readout, x_readout], dim=1)
        negative_pair = torch.cat([x_np_readout, x_readout], dim=1)
        real = torch.ones(positive_pair.shape[0], device=device)  
        fake = torch.zeros(negative_pair.shape[0], device=device)  
        score = (s_pp - s_np)
        perm = topk(score, self.ratio, batch)
        x = x_transform[perm] * self.non_lin(score[perm]).view(-1, 1)
        batch = batch[perm]
        filter_edge_index, filter_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, filter_edge_index, filter_edge_attr, batch, perm

In [11]:
from torch_geometric.nn import ASAPooling
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_CO(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_CO, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(CGIPool(channels, ratio=0.9))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm = self.pools[i - 1](x, edge_index, None, batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_CO(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

Seed: 042, Epoch: 010, Loss: 2.8205, Train Acc: 0.2963, Val Acc: 0.3084, Test Acc: 0.2757
Seed: 042, Epoch: 020, Loss: 2.3297, Train Acc: 0.4035, Val Acc: 0.4122, Test Acc: 0.3680
Seed: 042, Epoch: 030, Loss: 2.0551, Train Acc: 0.4566, Val Acc: 0.4540, Test Acc: 0.4031
Seed: 042, Epoch: 040, Loss: 1.8393, Train Acc: 0.5041, Val Acc: 0.4865, Test Acc: 0.4186
Seed: 042, Epoch: 050, Loss: 1.6957, Train Acc: 0.5352, Val Acc: 0.5162, Test Acc: 0.4551
Seed: 042, Epoch: 060, Loss: 1.6249, Train Acc: 0.5521, Val Acc: 0.5399, Test Acc: 0.4818
Seed: 042, Epoch: 070, Loss: 1.5724, Train Acc: 0.5626, Val Acc: 0.5419, Test Acc: 0.4788
Seed: 042, Epoch: 080, Loss: 1.5360, Train Acc: 0.5711, Val Acc: 0.5487, Test Acc: 0.4865
Seed: 042, Epoch: 090, Loss: 1.5195, Train Acc: 0.5749, Val Acc: 0.5599, Test Acc: 0.4941
Seed: 042, Epoch: 100, Loss: 1.4943, Train Acc: 0.5811, Val Acc: 0.5610, Test Acc: 0.4968
Seed: 042, Epoch: 110, Loss: 1.4771, Train Acc: 0.5832, Val Acc: 0.5623, Test Acc: 0.5025
Seed: 042,

### KMIS

In [23]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from typing import Callable, Optional, Tuple, Union
import torch
from torch.nn import Module
from torch_scatter import scatter, scatter_add, scatter_min
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
import torch
from torch_scatter import scatter_max, scatter_min
from torch_geometric.typing import Adj, OptTensor, SparseTensor, Tensor
from typing import Callable, Optional, Tuple, Union
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch.nn import Module
from torch_scatter import scatter_max, scatter_min
def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    r"""Returns a Maximal :math:`k`-Independent Set of a graph, i.e., a set of
    nodes (as a :class:`ByteTensor`) such that none of them are :math:`k`-hop
    neighbors, and any node in the graph has a :math:`k`-hop neighbor in the
    returned set.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method follows `Blelloch's Alogirithm
    <https://arxiv.org/abs/1202.3205>`_ for :math:`k = 1`, and its
    generalization by `Bacciu et al. <https://arxiv.org/abs/2208.03523>`_ for
    higher values of :math:`k`.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: :class:`ByteTensor`
    """
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1
    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)
    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()
    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  
        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()
        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  
        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n
    return mis
def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    r"""Computes the Maximal :math:`k`-Independent Set (:math:`k`-MIS)
    clustering of a graph, as defined in `"Generalizing Downsampling from
    Regular Data to Graphs" <https://arxiv.org/abs/2208.03523>`_.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method returns both the :math:`k`-MIS and the clustering, where the
    :math:`c`-th cluster refers to the :math:`c`-th element of the
    :math:`k`-MIS.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: (:class:`ByteTensor`, :class:`LongTensor`)
    """
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]
    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)
    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis
    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)
    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]
class KMISPooling(Module):
    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }
    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."
        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."
        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough
        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops
        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"
            self.lin = torch.nn.Linear(in_channels, 1)
    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x
        row, col, _ = adj.coo()
        x = x.view(-1)
        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()
        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)
        return x / k_sums
    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()
        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)
        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)
        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)
        if self.scorer == 'first':
            return x[..., [0]]
        if self.scorer == 'last':
            return x[..., [-1]]
        return self.scorer(x, edge_index, edge_attr, batch)
    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        """"""
        edge_index = edge_index.long()
        adj, n = edge_index, x.size(0)
        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))
        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)
        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)
        row, col, val = adj.coo()
        c = mis.sum()
        if val is None:
            val = torch.ones_like(row, dtype=torch.float)
        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)
        if self.remove_self_loops:
            adj = remove_diag(adj)
        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)
        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)
        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])
        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None
        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])
        if batch is not None:
            batch = batch[mis]
        perm = perm[mis]
        return x, edge_index, edge_attr, batch, mis, cluster, perm
    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""
        return f'{self.__class__.__name__}({channels}k={self.k})'

In [24]:
from torch_geometric.nn import ASAPooling
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(256, k=5, aggr_x='sum'))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_KMIS(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

Seed: 042, Epoch: 010, Loss: 3.0120, Train Acc: 0.2774, Val Acc: 0.3013, Test Acc: 0.2686
Seed: 042, Epoch: 020, Loss: 2.5648, Train Acc: 0.3541, Val Acc: 0.3868, Test Acc: 0.3409
Seed: 042, Epoch: 030, Loss: 2.1364, Train Acc: 0.4323, Val Acc: 0.4336, Test Acc: 0.3797
Seed: 042, Epoch: 040, Loss: 1.9311, Train Acc: 0.4804, Val Acc: 0.4747, Test Acc: 0.4109
Seed: 042, Epoch: 050, Loss: 1.7729, Train Acc: 0.5200, Val Acc: 0.5127, Test Acc: 0.4550
Seed: 042, Epoch: 060, Loss: 1.6788, Train Acc: 0.5392, Val Acc: 0.5205, Test Acc: 0.4574
Seed: 042, Epoch: 070, Loss: 1.6091, Train Acc: 0.5524, Val Acc: 0.5294, Test Acc: 0.4669
Seed: 042, Epoch: 080, Loss: 1.5656, Train Acc: 0.5651, Val Acc: 0.5417, Test Acc: 0.4758
Seed: 042, Epoch: 090, Loss: 1.5336, Train Acc: 0.5705, Val Acc: 0.5475, Test Acc: 0.4822
Seed: 042, Epoch: 100, Loss: 1.5167, Train Acc: 0.5768, Val Acc: 0.5544, Test Acc: 0.4880
Seed: 042, Epoch: 110, Loss: 1.4931, Train Acc: 0.5791, Val Acc: 0.5543, Test Acc: 0.4854
Seed: 042,

### GSAPool

In [25]:
import math
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, ChebConv, GraphConv
def uniform(size, tensor):
    if tensor is not None:
        bound = 1.0 / math.sqrt(size)
        tensor.data.uniform_(-bound, bound)
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))
def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = (x > scores_min).nonzero(as_tuple=False).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
             num_nodes.cumsum(dim=0)[:-1]], dim=0)
        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
        dense_x = x.new_full((batch_size * max_num_nodes, ),
                             torch.finfo(x.dtype).min)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)
        _, perm = dense_x.sort(dim=-1, descending=True)
        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)
        if isinstance(ratio, int):
            k = num_nodes.new_full((num_nodes.size(0), ), ratio)
            k = torch.min(k, num_nodes)
        else:
            k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)
        perm = perm[mask]
    return perm
def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i
    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]
    if edge_attr is not None:
        edge_attr = edge_attr[mask]
    return torch.stack([row, col], dim=0), edge_attr
class GSAPool(torch.nn.Module):
    def __init__(self, in_channels, pooling_ratio=0.5, alpha=0.6,
                        min_score=None, multiplier=1,
                        non_linearity=torch.tanh,
                        cus_drop_ratio =0):
        super(GSAPool,self).__init__()
        self.in_channels = in_channels
        self.ratio = pooling_ratio
        self.alpha = alpha
        self.sbtl_layer = GCNConv(in_channels,1)
        self.fbtl_layer = nn.Linear(in_channels, 1)
        self.fusion = GCNConv(in_channels,in_channels)
        self.min_score = min_score
        self.multiplier = multiplier
        self.fusion_flag = 0
        self.non_linearity = non_linearity
        self.dropout = torch.nn.Dropout(cus_drop_ratio)
    def conv_selection(self, conv, in_channels, conv_type=0):
        if(conv_type == 0):
            out_channels = 1
        elif(conv_type == 1):
            out_channels = in_channels
        if(conv == "GCNConv"):
            return GCNConv(in_channels,out_channels)
        elif(conv == "ChebConv"):
            return ChebConv(in_channels,out_channels,1)
        elif(conv == "SAGEConv"):
            return SAGEConv(in_channels,out_channels)
        elif(conv == "GATConv"):
            return GATConv(in_channels,out_channels, heads=1, concat=True)
        elif(conv == "GraphConv"):
            return GraphConv(in_channels,out_channels)
        else:
            raise ValueError
    def forward(self, x, edge_index, edge_attr=None, batch=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        score_s = self.sbtl_layer(x,edge_index).squeeze()
        score_f = self.fbtl_layer(x).squeeze()
        score = score_s*self.alpha + score_f*(1-self.alpha)
        score = score.unsqueeze(-1) if score.dim()==0 else score
        if self.min_score is None:
            score = self.non_linearity(score)
        else:
            score = softmax(score, batch)
        sc = self.dropout(score)
        perm = topk(sc, self.ratio, batch)
        if(self.fusion_flag == 1):
            x = self.fusion(x, edge_index)
        x_ae = x[perm]
        x = x[perm] * score[perm].view(-1, 1)
        x = self.multiplier * x if self.multiplier != 1 else x
        batch = batch[perm]
        edge_index, edge_attr = filter_adj(
            edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, edge_index, edge_attr, batch, perm, x_ae

In [26]:
from torch_geometric.nn import ASAPooling
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_GSA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_GSA, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.pools = torch.nn.ModuleList()
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(GSAPool(256, pooling_ratio=0.9, alpha = 0.6, cus_drop_ratio = 0))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.0, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x, edge_index, edge_attr, batch, perm, _ = self.pools[i - 1](x, edge_index, None, batch)
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_GSA(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

Seed: 042, Epoch: 010, Loss: 3.1351, Train Acc: 0.2226, Val Acc: 0.2992, Test Acc: 0.2945
Seed: 042, Epoch: 020, Loss: 2.5531, Train Acc: 0.3552, Val Acc: 0.3848, Test Acc: 0.3429
Seed: 042, Epoch: 030, Loss: 2.1509, Train Acc: 0.4393, Val Acc: 0.4487, Test Acc: 0.3909
Seed: 042, Epoch: 040, Loss: 1.9204, Train Acc: 0.4870, Val Acc: 0.4847, Test Acc: 0.4309
Seed: 042, Epoch: 050, Loss: 1.7517, Train Acc: 0.5223, Val Acc: 0.5113, Test Acc: 0.4510
Seed: 042, Epoch: 060, Loss: 1.6709, Train Acc: 0.5415, Val Acc: 0.5294, Test Acc: 0.4660
Seed: 042, Epoch: 070, Loss: 1.6126, Train Acc: 0.5539, Val Acc: 0.5405, Test Acc: 0.4756
Seed: 042, Epoch: 080, Loss: 1.5653, Train Acc: 0.5641, Val Acc: 0.5488, Test Acc: 0.4843
Seed: 042, Epoch: 090, Loss: 1.5510, Train Acc: 0.5673, Val Acc: 0.5489, Test Acc: 0.4833
Seed: 042, Epoch: 100, Loss: 1.5212, Train Acc: 0.5751, Val Acc: 0.5548, Test Acc: 0.4893
Seed: 042, Epoch: 110, Loss: 1.4951, Train Acc: 0.5793, Val Acc: 0.5569, Test Acc: 0.4932
Seed: 042,